In [56]:
import csv
import json

In [57]:
train_test_split = 0.8

In [58]:
# CSV columns: https://cloud.google.com/vision/automl/object-detection/docs/csv-format
# 1st col: TRAIN, VALIDATION, TEST, UNASSIGNED
# 2nd col: can use absolute path
# 2nd col: ball
# 3rd col: (x_relative_min, y_relative_min,,, x_relative_max, y_relative_max,,) with each point being within [0, 1]
header = ["train_test", "image_path", "label", "bounding_box"]

In [59]:
with open('imp/videos/labels_0/0.json') as label_file:
    data = json.load(label_file)

In [60]:
frame_height = data["-1"]["frameHeight"]
frame_width = data["-1"]["frameWidth"]
# frame_count = data["-1"]["frameCount"]
ball_radius = data["-1"]["ballRadius"]


In [61]:
def find_bounds(x, y):
    x_min = x - ball_radius
    x_max = x + ball_radius
    y_min = y - ball_radius
    y_max = y + ball_radius
    if x_min < 0:
        x_max -= x_min
        x_min = 0
    elif x_max >= frame_width:
        x_min -= (x_max - frame_width)
        x_max = frame_width
    if y_min < 0:
        y_max -= y_min
        y_min = 0
    elif y_max >= frame_height:
        y_min -= (y_max - frame_height)
        y_max = frame_height
    # normalize coordinates between [0, 1]
    x_min = x_min/frame_width
    x_max = x_max/frame_width
    y_min = y_min/frame_height
    y_max = y_max/frame_height
    return x_min, y_min, "", "", x_max, y_max, "", ""

In [62]:
df = []
frame_nums = data.keys()
frame_count = len(frame_nums)
for i, k in enumerate(frame_nums):
    if "ballCoord" not in data[k]:
        continue
    bounding_box = find_bounds(*data[k]["ballCoord"])
    set_type = "TRAIN" if i < frame_count * train_test_split else "VALIDATION"
    df.append([set_type, f"{k}.jpeg", "ball", *bounding_box])

In [63]:
with open('imp/videos/data/0.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(df)